In [1]:
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor
import warnings

from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier,CatBoostRegressor
from xgboost import XGBClassifier
import warnings

In [2]:
data = pd.read_csv('train.csv',index_col=0)
data.columns

Index(['Name', 'Gender', 'Age', 'City', 'Working Professional or Student',
       'Profession', 'Academic Pressure', 'Work Pressure', 'CGPA',
       'Study Satisfaction', 'Job Satisfaction', 'Sleep Duration',
       'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?',
       'Work/Study Hours', 'Financial Stress',
       'Family History of Mental Illness', 'Depression'],
      dtype='object')

In [3]:
data.isnull().sum()

Name                                          0
Gender                                        0
Age                                           0
City                                          0
Working Professional or Student               0
Profession                                36630
Academic Pressure                        112803
Work Pressure                             27918
CGPA                                     112802
Study Satisfaction                       112803
Job Satisfaction                          27910
Sleep Duration                                0
Dietary Habits                                4
Degree                                        2
Have you ever had suicidal thoughts ?         0
Work/Study Hours                              0
Financial Stress                              4
Family History of Mental Illness              0
Depression                                    0
dtype: int64

In [4]:
df = data.drop(columns=['Name','City','Degree'],axis=1)


In [5]:
df.head(3)

,Gender,Age,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
id,,,,,,,,,,,,,,,,
0,Female,49.0,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,No,1.0,2.0,No,0
1,Male,26.0,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,Yes,7.0,3.0,No,1
2,Male,33.0,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,Yes,3.0,1.0,No,1


In [6]:
df.columns

Index(['Gender', 'Age', 'Working Professional or Student', 'Profession',
       'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction',
       'Job Satisfaction', 'Sleep Duration', 'Dietary Habits',
       'Have you ever had suicidal thoughts ?', 'Work/Study Hours',
       'Financial Stress', 'Family History of Mental Illness', 'Depression'],
      dtype='object')

In [7]:
df['Sleep Duration'].value_counts()

Sleep Duration
Less than 5 hours    38784
7-8 hours            36969
More than 8 hours    32726
5-6 hours            32142
3-4 hours               12
6-7 hours                8
4-5 hours                7
2-3 hours                5
4-6 hours                5
6-8 hours                4
1-6 hours                4
No                       4
9-11 hours               2
10-11 hours              2
Sleep_Duration           2
Unhealthy                2
45                       2
8-9 hours                2
10-6 hours               1
9-5                      1
45-48 hours              1
3-6 hours                1
Work_Study_Hours         1
49 hours                 1
than 5 hours             1
Pune                     1
9-6 hours                1
8 hours                  1
35-36 hours              1
Indore                   1
1-3 hours                1
55-66 hours              1
Moderate                 1
40-45 hours              1
1-2 hours                1
9-5 hours                1
Name: count, 

In [8]:
sleep_duration_map = {
    'Less than 5 hours': 4,
    '7-8 hours': 7.5,
    'More than 8 hours': 9,
    '5-6 hours': 5.5,
    '3-4 hours': 3.5,
    '6-7 hours': 6.5,
    '4-5 hours': 4.5,
    '2-3 hours': 2.5,
    '4-6 hours': 5,
    '6-8 hours': 7,
    '1-6 hours': 3.5,
    'No': 0,  # Assuming 'No' refers to no sleep recorded
    '9-11 hours': 10,
    '10-11 hours': 10.5,
    'Sleep_Duration': 9,
    'Unhealthy': 4,  # Assuming this value refers to a state of sleep duration, could adjust based on context
    '45': 5,
    '8-9 hours': 8.5,
    '10-6 hours': 8,
    '9-5': 7,
    '45-48 hours': 47,
    '3-6 hours': 4.5,
    'Work_Study_Hours': 6,
    '49 hours': 48,
    'than 5 hours': 4.5,
    'Pune': 5,  # These may need manual handling depending on context
    '9-6 hours': 7.5,
    '8 hours': 8,
    '35-36 hours': 35.5,
    'Indore': 5,
    '1-3 hours': 2,
    '55-66 hours': 60,
    'Moderate': 5,
    '40-45 hours': 42.5,
    '1-2 hours': 1.5,
    '9-5 hours': 7.5
}

df['Sleep Duration'] = df['Sleep Duration'].map(sleep_duration_map)


In [9]:
df.columns

Index(['Gender', 'Age', 'Working Professional or Student', 'Profession',
       'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction',
       'Job Satisfaction', 'Sleep Duration', 'Dietary Habits',
       'Have you ever had suicidal thoughts ?', 'Work/Study Hours',
       'Financial Stress', 'Family History of Mental Illness', 'Depression'],
      dtype='object')

In [10]:
df.drop(columns=['Profession','CGPA','Sleep Duration', 'Dietary Habits'],axis=1,inplace=True)

In [11]:
y_train = df[['Depression']]
X_train = df.drop('Depression',axis=1)
X_train.columns

Index(['Gender', 'Age', 'Working Professional or Student', 'Academic Pressure',
       'Work Pressure', 'Study Satisfaction', 'Job Satisfaction',
       'Have you ever had suicidal thoughts ?', 'Work/Study Hours',
       'Financial Stress', 'Family History of Mental Illness'],
      dtype='object')

In [12]:
ohe = OneHotEncoder(handle_unknown='ignore',sparse_output=False,drop='first').set_output(transform='pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude=object) ),
                               (ohe,make_column_selector(dtype_include=object) ) ).set_output(transform='pandas')

simple1 = SimpleImputer(strategy='mean').set_output(transform='pandas')
simple2 = SimpleImputer(strategy='constant',fill_value='unknown').set_output(transform='pandas')
ct_i = make_column_transformer((simple1,make_column_selector(dtype_exclude=object) ),
                               (simple2,make_column_selector(dtype_include=object) ) ).set_output(transform='pandas')

In [13]:
lr = LogisticRegression(random_state = 24)
pipe = Pipeline([('IMP',ct_i),('OHE',ct),('LR',lr)])


In [14]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 24)
params = {'LR__solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga'],
         'LR__C': np.linspace(0.001,1,11)}

In [15]:
gcv = GridSearchCV(pipe,param_grid=params,cv= kfold,scoring = 'roc_auc',n_jobs=-1,verbose=2)


In [16]:
gcv.fit(X_train,y_train)

Fitting 5 folds for each of 66 candidates, totalling 330 fits


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('IMP',
                                        ColumnTransformer(transformers=[('simpleimputer-1',
                                                                         SimpleImputer(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000027249F13020>),
                                                                        ('simpleimputer-2',
                                                                         SimpleImputer(fill_value='unknown',
                                                                                       strategy='constant'),
                                                                         <sklearn.compose...
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000027249F13680>)])),
                                       ('LR',
                                        LogisticRegression(random_state=24))]),
             n_jobs=-1,
             param_grid={'LR__C': array([0.001 , 0.1009, 0.2008, 0.3007, 0.4006, 0.5005, 0.6004, 0.7003,
       0.8002, 0.9001, 1.    ]),
                         'LR__solver': ['lbfgs', 'liblinear', 'newton-cg',
                                        'newton-cholesky', 'sag', 'saga']},
             scoring='roc_auc', verbose=2)

In [17]:
print(gcv.best_params_)
print(gcv.best_score_)

{'LR__C': 0.1009, 'LR__solver': 'lbfgs'}
0.9716116892808435


In [18]:
X_test = pd.read_csv("test.csv",index_col = 0)
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93800 entries, 140700 to 234499
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Name                                   93800 non-null  object 
 1   Gender                                 93800 non-null  object 
 2   Age                                    93800 non-null  float64
 3   City                                   93800 non-null  object 
 4   Working Professional or Student        93800 non-null  object 
 5   Profession                             69168 non-null  object 
 6   Academic Pressure                      18767 non-null  float64
 7   Work Pressure                          75022 non-null  float64
 8   CGPA                                   18766 non-null  float64
 9   Study Satisfaction                     18767 non-null  float64
 10  Job Satisfaction                       75026 non-null  float64
 11  S

In [19]:
bm_gb = gcv.best_estimator_
y_pred = bm_gb.predict(X_test)

In [20]:
sample = pd.read_csv('sample_submission.csv')

In [21]:

sample['Depression'] = y_pred
sample.to_csv('Classification_nov_16.csv',index=False)

## random Forest

In [23]:
rbf = RandomForestClassifier(random_state=24)
ohe = OneHotEncoder(handle_unknown='ignore',sparse_output=False,drop='first').set_output(transform='pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude=object) ),
                               (ohe,make_column_selector(dtype_include=object) ) ).set_output(transform='pandas')

simple1 = SimpleImputer(strategy='mean').set_output(transform='pandas')
simple2 = SimpleImputer(strategy='constant',fill_value='unknown').set_output(transform='pandas')
ct_i = make_column_transformer((simple1,make_column_selector(dtype_exclude=object) ),
                               (simple2,make_column_selector(dtype_include=object) ) ).set_output(transform='pandas')

In [24]:
pipe = Pipeline([('IMP',ct_i),('OHE',ct),('RBF',rbf)])


In [25]:
pipe.get_params()

{'memory': None,
 'steps': [('IMP',
   ColumnTransformer(transformers=[('simpleimputer-1', SimpleImputer(),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000002724CE1E870>),
                                   ('simpleimputer-2',
                                    SimpleImputer(fill_value='unknown',
                                                  strategy='constant'),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000002724CE1C050>)])),
  ('OHE',
   ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000002724CDC83B0>),
                                   ('onehotencoder',
                                    OneHotEncoder(drop='first',
                                                  handle_unknown='ignore',
                                            

In [26]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 24)
params = {'RBF__min_samples_split':[2,10,25,50],
          'RBF__min_samples_leaf':[1,10,25,50],
          'RBF__max_depth':[None,4,6],
          'RBF__max_features':[2,3,4,5,6,None]
         }

In [ ]:
gcv = GridSearchCV(pipe,param_grid=params,cv= kfold,scoring = 'roc_auc',n_jobs=-1,verbose=2)
gcv.fit(X_train,y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


In [ ]:
print(gcv.best_params_)
print(gcv.best_score_)

In [ ]:
X_test = pd.read_csv("test.csv",index_col = 0)
X_test.info()

In [ ]:
bm_gb = gcv.best_estimator_
y_pred = bm_gb.predict(X_test)

In [ ]:
sample = pd.read_csv('sample_submission.csv')

sample['Depression'] = y_pred
sample.to_csv('Random_Forest_nov_16.csv',index=False)

In [ ]:
cgbm = CatBoostRegressor(random_state=24, cat_features=list(X.columns[X.dtypes==object])) #can give like this -> ))
params = {
         'n_estimators': [10,20,30,50,100,200],
         'max_depth': [2,3,4],
         'learning_rate': np.linspace(0.001,1,6)         
}
          
          
         

kfold = KFold(n_splits = 5,shuffle = True,random_state =24)
